In this exercise, you'll build a project management assistant using OpenAI API Function Calling
A .csv file is used to simulate reading and writing from a database or project management tool API.  Follow the directions in the starter code below, and try to build the functions and function calling logic before you look at the solution on the next page! 

Import necessary libraries

In [1]:
import openai
openai.api_base = "https://openai.vocareum.com/v1"
import json
import pandas as pd

# OpenAI key here.
openai.api_key = "voc-605570984126677350407866e29880939962.25846762"

First, define the Python functions that will read and write from the project_management.csv file using Pandas dataframes. This code uses Pandas dataframes to read and write from the .csv file. 

We define 3 tasks our project management assistant can perform. 

Each function returns a JSON string as output

In [2]:
# Load the project management data
df = pd.read_csv('project_management.csv')

def task_retrieval_and_status_updates(task_id, status, last_updated):
    """Retrieve and update task status"""
    df.loc[df['Task ID'] == task_id, 'Status'] = status
    df.loc[df['Task ID'] == task_id, 'Last Updated'] = last_updated
    df.to_csv('project_management.csv', index=False)  # save changes to file
    task = df.loc[df['Task ID'] == task_id]
    return json.dumps(task.to_dict())

def project_reporting_and_analytics(project_id):
    """Generate reports on project progress and team performance"""
    project = df.loc[df['Project ID'] == project_id]
    return json.dumps(project.to_dict())

def resource_allocation_and_scheduling(task_id, assigned_to, time_estimate, due_date, status):
    """Allocate tasks based on current workloads and schedules"""
    df.loc[df['Task ID'] == task_id, 'Assigned To'] = assigned_to
    df.loc[df['Task ID'] == task_id, 'Time Estimate'] = time_estimate
    df.loc[df['Task ID'] == task_id, 'Due Date'] = due_date
    df.loc[df['Task ID'] == task_id, 'Status'] = status
    df.to_csv('project_management.csv', index=False)  # save changes to file
    return json.dumps(task.to_dict())

Next, we'll build the project management assistant conversation. 

We'll define the messages to send to the model, including a tools dictionary that defines a list of tools, which are the functions that are available to the model to identify and parse parameters for. 

In [8]:
def run_conversation():
    messages = [
        {"role": "system", "content": "You are a project management assistant..."},
        {"role": "user", "content": "Change the status of task 2 to completed."}
    ]
    
    tools = [
        {
            "type": "function",
            "function": {
                "name": "task_retrieval_and_status_updates",
                "description": "Retrieve and update task status",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "task_id": {"type": "integer", "description": "The unique identifier for the task"},
                        "status": {"type": "string", "description": "The new status of the task"},
                        "last_updated": {"type": "string", "description": "The date of the last status update"}
                    },
                    "required": ["task_id", "status", "last_updated"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "project_reporting_and_analytics",
                "description": "Generate a project report and analytics summary",
                "parameters": {
                    "type": "object",
                    "properties": {}
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "resource_allocation_and_scheduling",
                "description": "Reassign tasks to team members",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "task_id": {"type": "integer", "description": "The unique identifier for the task"},
                        "new_assignee": {"type": "string", "description": "The new person assigned to the task"}
                    },
                    "required": ["task_id", "new_assignee"]
                }
            }
        }
    ]
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response['choices'][0]['message']
    tool_calls = response_message.get('tool_calls')
    
    if tool_calls:
        available_functions = {
            "task_retrieval_and_status_updates": task_retrieval_and_status_updates,
            "project_reporting_and_analytics": project_reporting_and_analytics,
            "resource_allocation_and_scheduling": resource_allocation_and_scheduling
        }
        
        messages.append(response_message)  # Extend conversation with first response
        
        # Iterate through the tool calls in the response
        for tool_call in tool_calls:
            function_name = tool_call['function']['name']
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call['function']['arguments'])
            
            # Call the appropriate function and pass the necessary arguments
            if function_name == 'task_retrieval_and_status_updates':
                function_response = function_to_call(
                    task_id=function_args.get("task_id"),
                    status=function_args.get("status"),
                    last_updated=function_args.get("last_updated")
                )
            elif function_name == 'project_reporting_and_analytics':
                function_response = function_to_call()
            elif function_name == 'resource_allocation_and_scheduling':
                function_response = function_to_call(
                    task_id=function_args.get("task_id"),
                    new_assignee=function_args.get("new_assignee")
                )
            
            # Include tool_call_id in the response message
            message_to_append = {
                "tool_call_id": tool_call['id'],  # Include the correct tool_call_id here
                "role": "tool",
                "name": function_name,
                "content": function_response
            }
            
            messages.append(message_to_append)  # Extend conversation with function response

        # Send the updated conversation back to the model for a new response
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-1106",
            messages=messages
        )
        
        return second_response


In [9]:
print(run_conversation()) # will print the second response from the model

{
  "id": "chatcmpl-AEUndgjxa8x1bcktTjZyosjrqC9Te",
  "object": "chat.completion",
  "created": 1728018349,
  "model": "gpt-3.5-turbo-1106",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The status of task 2, \"Implement Login Page,\" has been successfully updated to completed as of May 15, 2022.",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 239,
    "completion_tokens": 28,
    "total_tokens": 267,
    "prompt_tokens_details": {
      "cached_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0
    }
  },
  "system_fingerprint": "fp_0338b7694d"
}
